In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

np.random.seed(42)

from sklearn.model_selection import train_test_split
import lightgbm as lgbm
from sklearn.metrics import confusion_matrix, accuracy_score

# 1. 데이터 로드

In [2]:
train=pd.read_csv('input/train.csv', index_col=0) #인덱스 col=0도 check!
test=pd.read_csv('input/test_x.csv', index_col=0)
submission=pd.read_csv('input/sample_submission.csv', index_col=0)  

In [22]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

In [25]:
interactions = pd.DataFrame(data=poly.fit_transform(train[['QaE','QbE']]), columns=poly.get_feature_names(['QaE','QbE']))
interactions.drop(['QaE','QbE'], axis=1, inplace=True) 
interactions

,QaE^2,QaE QbE,QbE^2
0,131769.0,497310.0,1876900.0
1,418609.0,849511.0,1723969.0
2,2634129.0,2402040.0,2190400.0
3,254016.0,1164744.0,5340721.0
4,859329.0,655389.0,499849.0
...,...,...,...
45527,1102500.0,649950.0,383161.0
45528,337561.0,786093.0,1830609.0
45529,351649.0,508201.0,734449.0
45530,558009.0,994257.0,1771561.0


In [26]:
train = pd.concat([train, interactions], axis=1)
train

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13,QaE^2,QaE QbE,QbE^2
0,3.0,363,4.0,1370,5.0,997,1.0,1024,2.0,1577,...,1,1,0,1,0,1,1,131769.0,497310.0,1876900.0
1,5.0,647,5.0,1313,3.0,3387,5.0,2969,1.0,4320,...,1,1,0,1,0,1,1,418609.0,849511.0,1723969.0
2,4.0,1623,1.0,1480,1.0,1021,4.0,3374,5.0,1333,...,1,1,1,1,0,1,1,2634129.0,2402040.0,2190400.0
3,3.0,504,3.0,2311,4.0,992,3.0,3245,1.0,357,...,0,1,0,1,0,1,1,254016.0,1164744.0,5340721.0
4,1.0,927,1.0,707,5.0,556,2.0,1062,1.0,1014,...,1,1,0,1,1,1,1,859329.0,655389.0,499849.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45527,2.0,1050,5.0,619,4.0,328,1.0,285,1.0,602,...,1,1,1,1,0,1,1,1102500.0,649950.0,383161.0
45528,2.0,581,3.0,1353,4.0,1164,1.0,798,3.0,1680,...,1,1,1,1,0,1,1,337561.0,786093.0,1830609.0
45529,4.0,593,1.0,857,1.0,1047,4.0,1515,5.0,1690,...,1,1,0,1,0,1,1,351649.0,508201.0,734449.0
45530,1.0,747,3.0,1331,4.0,892,2.0,1281,1.0,1328,...,1,1,1,1,0,1,1,558009.0,994257.0,1771561.0


# 3. 모델 학습

In [3]:
from pycaret.classification import *

In [4]:
set_config('seed', 42)

In [6]:
import random
random.seed(42)

In [7]:
import gc
gc.collect()

44

In [27]:
%%time
clf = setup(session_id=42, 
            data = train, target = 'voted'#, polynomial_features=True
            #,feature_selection=True
            ,use_gpu=True
           )

Setup Succesfully Completed!


,Description,Value
0,session_id,42
1,Target Type,Binary
2,Label Encoded,"1: 0, 2: 1"
3,Original Data,"(45532, 80)"
4,Missing Values,False
5,Numeric Features,44
6,Categorical Features,35
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


Wall time: 40 s


In [28]:
#best_3 = compare_models(sort = 'AUC', n_select = 3)
best_5 = compare_models(sort = 'AUC', n_select = 2)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,CatBoost Classifier,0.6926,0.7635,0.6550,0.7511,0.6997,0.3878,0.3917,43.8155
1,Gradient Boosting Classifier,0.6945,0.7634,0.6358,0.7658,0.6947,0.3942,0.4011,31.0396
2,Light Gradient Boosting Machine,0.6908,0.7622,0.6410,0.7564,0.6939,0.3858,0.3912,1.7807
3,Linear Discriminant Analysis,0.6901,0.7607,0.6603,0.7442,0.6997,0.3820,0.3850,1.5829
4,Extra Trees Classifier,0.6867,0.7562,0.6419,0.7494,0.6914,0.3772,0.3819,6.7414
5,Ada Boost Classifier,0.6888,0.7557,0.6488,0.7486,0.6951,0.3807,0.3848,7.6010
6,Extreme Gradient Boosting,0.6705,0.7411,0.6578,0.7164,0.6858,0.3407,0.3421,22.8639
7,Random Forest Classifier,0.6561,0.7109,0.6051,0.7211,0.6580,0.3174,0.3223,0.6427
8,Decision Tree Classifier,0.6119,0.6087,0.6436,0.6457,0.6446,0.2173,0.2173,2.4646
9,Logistic Regression,0.4533,0.5340,0.0003,0.1667,0.0006,0.0001,0.0007,0.6122


In [16]:
blended = blend_models(estimator_list = best_5, fold = 3, method = 'soft', optimize='AUC')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6906,0.7625,0.6408,0.7562,0.6937,0.3854,0.3908
1,0.6949,0.7631,0.6478,0.7590,0.6990,0.3937,0.3987
2,0.6938,0.7664,0.6425,0.7604,0.6965,0.3919,0.3976
Mean,0.6931,0.7640,0.6437,0.7585,0.6964,0.3903,0.3957
SD,0.0018,0.0017,0.0030,0.0017,0.0021,0.0035,0.0035


In [ ]:
pred_holdout = predict_model(blended)

In [ ]:
final_model = finalize_model(blended)

In [ ]:
predictions = predict_model(final_model, data = test)

# 4. 예측

In [ ]:
submission['voted'] = predictions['Score']

# 5. 제출

In [ ]:
submission.to_csv('output/20201001-1-O-all.csv')